In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import nltk  
import re 
from string import punctuation
from nltk import word_tokenize
from nltk.tokenize import WordPunctTokenizer, PunktSentenceTokenizer
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

import string
import os
import pickle

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import Input, Embedding, LSTM, Dense

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\43664\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\43664\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\43664\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\43664\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ImportError: cannot import name 'pad_sequences' from 'keras.preprocessing.sequence' (C:\Users\43664\anaconda3\lib\site-packages\keras\preprocessing\sequence.py)

## Read and inspect data

In [2]:
raw_train_data = pd.read_csv('train.csv')
raw_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404290 non-null  int64 
 1   qid1          404290 non-null  int64 
 2   qid2          404290 non-null  int64 
 3   question1     404289 non-null  object
 4   question2     404288 non-null  object
 5   is_duplicate  404290 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [96]:
raw_train_data.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [97]:
raw_train_data.dtypes

id               int64
qid1             int64
qid2             int64
question1       object
question2       object
is_duplicate     int64
dtype: object

In [4]:
raw_test_data = pd.read_csv('test.csv')
raw_test_data.head(5)

C:\Users\43664\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [98]:
print(raw_train_data.isnull().sum())

id              0
qid1            0
qid2            0
question1       1
question2       2
is_duplicate    0
dtype: int64


In [5]:
raw_test_data.isnull().sum()

test_id      0
question1    4
question2    6
dtype: int64

## Data Preparation

In [3]:
train_data = raw_train_data.dropna(axis=0)
#test_data = raw_test_data.dropna(axis=0)

In [100]:
train_data.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [9]:
test_data.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [4]:
train_data = train_data.drop(['id', 'qid1', 'qid2'], axis=1)
train_data.head(5)

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [104]:
train_data['question1'].iloc[0]

'What is the step by step guide to invest in share market in india?'

In [8]:
train_copy = train_data.copy()

## Text Preparation

In [9]:
## REMOVE ACCENTED CHARACTERS Sómě Áccěntěd těxt ##
import unicodedata

def remove_accented_chars_func(text):
    return unicodedata.normalize('NFKD', str(text)).encode('ascii', 'ignore').decode('utf-8', 'ignore')

## REMOVE PUNCTUATION ##

def remove_punctuation_func(text):
    return re.sub(r'[^a-zA-Z0-9]', ' ', str(text))

## REMOVE IRRELEVANT CHARACTERS 

def remove_irr_char_func(text):
    return re.sub(r'[^a-zA-Z]', ' ', str(text))

## REMOVE EXTRA WHITESPACEs ##

def remove_extra_whitespaces_func(text):
    return re.sub(r'^\s*|\s\s*', ' ', str(text)).strip()

## COUNT OF STRINGS ##

def word_count_func(text):
    return len(str(text).split())

## REMOVE HASHTAG IN FRONT OF A TEXT##

def remove_hashtag(text):
    return re.sub(r'#([^\s]+)', r'\1', str(text))

In [11]:
train_copy['question1'] = train_copy['question1'].str.lower()
train_copy['question2'] = train_copy['question2'].str.lower()

train_copy['question1'] = train_copy['question1'].apply(remove_accented_chars_func)
train_copy['question2'] = train_copy['question2'].apply(remove_accented_chars_func)

train_copy['question1'] = train_copy['question1'].apply(remove_punctuation_func)
train_copy['question2'] = train_copy['question2'].apply(remove_punctuation_func)

train_copy['question1'] = train_copy['question1'].apply(remove_irr_char_func)
train_copy['question2'] = train_copy['question2'].apply(remove_irr_char_func)

train_copy['question1'] = train_copy['question1'].apply(remove_extra_whitespaces_func)
train_copy['question2'] = train_copy['question2'].apply(remove_extra_whitespaces_func)

train_copy['question1'] = train_copy['question1'].apply(remove_hashtag)
train_copy['question2'] = train_copy['question2'].apply(remove_hashtag)

train_copy.head(5)

KeyboardInterrupt: 

## Split data

In [15]:
features = ['question1', 'question2']

X_data = train_copy[features]
X_data.head()

,question1,question2
0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...
1,what is the story of kohinoor koh i noor diamond,what would happen if the indian government sto...
2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...
3,why am i mentally very lonely how can i solve it,find the remainder when math math is divided by
4,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water


In [16]:
y_data = train_copy['is_duplicate']
y_data.head(5)

0    0
1    0
2    0
3    0
4    0
Name: is_duplicate, dtype: int64

In [18]:
# Split the set into train and split data into 60/40, random_state 0 

from sklearn.model_selection import train_test_split
X_train, X_split, y_train, y_split = train_test_split(X_data, y_data, test_size=0.99, random_state = 0)  ## cambiar test size

print("Whole dataset: ", len(train_copy))
print("X_train size =", len(X_train))
print("X_split size =", len(X_split))
print("y_train size =", len(y_train))
print("y_split size =", len(y_split))

Whole dataset:  404287
X_train size = 4042
X_split size = 400245
y_train size = 4042
y_split size = 400245


In [19]:
# Split the set into test and validation into 50/50 (from 40% of the splitted data), random_state 0

from sklearn.model_selection import train_test_split
X_test, X_val, y_test, y_val = train_test_split(X_split, y_split, test_size=0.98, random_state = 0)

print("Whole dataset: ", len(train_copy))
print("X_test size =", len(X_test))
print("X_val size =", len(X_val))
print("y_test size =", len(y_test))
print("y_val size =", len(y_val))

Whole dataset:  404287
X_test size = 8004
X_val size = 392241
y_test size = 8004
y_val size = 392241


In [143]:
X_val.head(5)

,question1,question2
189347,what are the most asked mainframe interview qu...,what is the most asked question in an interview
327140,how long should i wait for my bitcoin core wal...,can one set up a bitcoin wallet under false id...
126940,i got selected for internship at inria th june...,i got selected for internship at inria th june...
78406,how do you stay fit if you don t like the gym,how can i stay fit with out going gym
372855,what is the best budget hotels in cochin for a...,what is the best budget hotels in mussoorie fo...


## Text Normalization

## Tokenization

In [36]:
### X_train ###
## QUESTION 1 ##
X_train_text_q1 = X_train['question1'] 

X_train_q1 = []

for sent_train_q1 in X_train_text_q1:
    
    #lowercase
    #sent = sent.lower()
  
    sent_train_q1 = sent_train_q1.translate(str.maketrans('', '', string.punctuation))

  ## tokenize ##
    train_tokens_q1 = nltk.word_tokenize(sent_train_q1)

  ## remove stop-words ##
    english_stopwords = stopwords.words('english')
    #english_stopwords = english_stopwords + ['did', 'could', 'would', 'have', 'had', 'has', 'might', 'should', 'was', 'were', 'does', 'much', 'br', 'n']
    train_tokens_q1 = [el for el in train_tokens_q1 if not el in english_stopwords]
    
 ## stemming ##

    #stemmed_tokens = [PorterStemmer().stem(el) for el in train_tokens_q1]
    
    
  ## add to output list ##
    X_train_q1.append(" ".join(train_tokens_q1))

In [40]:
### X_train ###
## QUESTION 2 ##
X_train_text_q2 = X_train['question2'] 

X_train_q2 = []

for sent_train_q2 in X_train_text_q2:
    
    #lowercase
    #sent = sent.lower()
  
    sent_train_q2 = sent_train_q2.translate(str.maketrans('', '', string.punctuation))

  ## tokenize ##
    train_tokens_q2 = nltk.word_tokenize(sent_train_q2)

  ## remove stop-words ##
    english_stopwords = stopwords.words('english')
    #english_stopwords = english_stopwords + ['did', 'could', 'would', 'have', 'had', 'has', 'might', 'should', 'was', 'were', 'does', 'much', 'br', 'n']
    train_tokens_q2 = [el for el in train_tokens_q2 if not el in english_stopwords]

   ## stemming ##

    #stemmed_tokens = [PorterStemmer().stem(el) for el in train_tokens_q2]

  ## add to output list ##
    X_train_q2.append(" ".join(train_tokens_q2))

In [66]:
### X_test ###
## QUESTION 1 ##
X_test_text_q1 = X_test['question1'] 

X_test_q1 = []

for sent_test_q1 in X_test_text_q1:
    
    #lowercase
    #sent = sent.lower()
  
    sent_test_q1 = sent_test_q1.translate(str.maketrans('', '', string.punctuation))

  ## tokenize ##
    test_tokens_q1 = nltk.word_tokenize(sent_test_q1)

  ## remove stop-words ##
    english_stopwords = stopwords.words('english')
    #english_stopwords = english_stopwords + ['did', 'could', 'would', 'have', 'had', 'has', 'might', 'should', 'was', 'were', 'does', 'much', 'br', 'n']
    test_tokens_q1 = [el for el in test_tokens_q1 if not el in english_stopwords]

  ## lemmatization ##
 
    #test_tokens_lemma_q1 = [wnl.lemmatize(el, get_wordnet_pos(nltk.pos_tag([el])[0][1])) for el in test_tokens_q1]
    
    ## stemming ##

    #stemmed_tokens = [PorterStemmer().stem(el) for el in test_tokens_q1]


  ## add to output list ##
    X_test_q1.append(" ".join(test_tokens_q1))

In [74]:
### X_test ###
## QUESTION 2 ##
X_test_text_q2 = X_test['question2'] 

X_test_q2 = []

for sent_test_q2 in X_test_text_q2:
    
    #lowercase
    #sent = sent.lower()
  
    sent_test_q2 = sent_test_q2.translate(str.maketrans('', '', string.punctuation))

  ## tokenize ##
    test_tokens_q2 = nltk.word_tokenize(sent_test_q2)

  ## remove stop-words ##
    english_stopwords = stopwords.words('english')
    #english_stopwords = english_stopwords + ['did', 'could', 'would', 'have', 'had', 'has', 'might', 'should', 'was', 'were', 'does', 'much', 'br', 'n']
    test_tokens_q2 = [el for el in test_tokens_q2 if not el in english_stopwords]

  ## lemmatization ##
 
    #test_tokens_lemma_q2 = [wnl.lemmatize(el, get_wordnet_pos(nltk.pos_tag([el])[0][1])) for el in test_tokens_q2]
    
    ## stemming ##

    #stemmed_tokens = [PorterStemmer().stem(el) for el in test_tokens_q2]


  ## add to output list ##
    X_test_q2.append(" ".join(test_tokens_q2))

In [160]:
### X_val ###
## QUESTION 1 ##
X_val_text_q1 = X_val['question1'] 

X_val_q1 = []

for sent_val_q1 in X_val_text_q1:
    
    #lowercase
    #sent = sent.lower()
  
    sent_val_q1 = sent_val_q1.translate(str.maketrans('', '', string.punctuation))

  ## tokenize ##
    val_tokens_q1 = nltk.word_tokenize(sent_val_q1)

  ## remove stop-words ##
    english_stopwords = stopwords.words('english')
    #english_stopwords = english_stopwords + ['did', 'could', 'would', 'have', 'had', 'has', 'might', 'should', 'was', 'were', 'does', 'much', 'br', 'n']
    val_tokens_q1 = [el for el in val_tokens_q1 if not el in english_stopwords]

  ## lemmatization ##
 
    #val_tokens_lemma_q1 = [wnl.lemmatize(el, get_wordnet_pos(nltk.pos_tag([el])[0][1])) for el in val_tokens_q1]
    
    ## stemming ##

    #stemmed_tokens = [PorterStemmer().stem(el) for el in val_tokens_q1]


  ## add to output list ##
    X_val_q1.append(" ".join(val_tokens_q1))

In [162]:
### X_val ###
## QUESTION 2 ##
X_val_text_q2 = X_val['question2'] 

X_val_q2 = []

for sent_val_q2 in X_val_text_q2:
    
    #lowercase
    #sent = sent.lower()
  
    sent_val_q2 = sent_val_q2.translate(str.maketrans('', '', string.punctuation))

  ## tokenize ##
    val_tokens_q2 = nltk.word_tokenize(sent_val_q2)

  ## remove stop-words ##
    english_stopwords = stopwords.words('english')
    #english_stopwords = english_stopwords + ['did', 'could', 'would', 'have', 'had', 'has', 'might', 'should', 'was', 'were', 'does', 'much', 'br', 'n']
    val_tokens_q2 = [el for el in val_tokens_q2 if not el in english_stopwords]

  ## lemmatization ##
 
    #val_tokens_lemma_q2 = [wnl.lemmatize(el, get_wordnet_pos(nltk.pos_tag([el])[0][1])) for el in val_tokens_q2]
    
    ## stemming ##

    #stemmed_tokens = [PorterStemmer().stem(el) for el in val_tokens_q2]


  ## add to output list ##
    X_val_q2.append(" ".join(val_tokens_q2))

In [61]:
df_q1 = pd.DataFrame(X_train_q1, columns=['question1'])
df_q2 = pd.DataFrame(X_train_q2, columns=['question2'])
train_df = df_q1.join(df_q2, lsuffix='_left', rsuffix='_right')
train_df

,question1,question2
0,begin take donald trump elected president thin...,donald trump potential good president
1,many legs butterflies used,ok chest legs day squat use leg press
2,true sweden largest number rape case europe,true sweden became rape capital europe
3,revenue model socialtrade biz,revenue model openbay
4,net protector antivirus good comparison quickh...,urgent net protector antivirus ran windows vul...
...,...,...
4037,java android,r java android
4038,proper length men pants,proper length men suit pants
4039,know whether want mbbs something else,know really want become doctor
4040,hack snapchat chat history,spy snapchat chat history


In [75]:
df_q1 = pd.DataFrame(X_test_q1, columns=['question1'])
df_q2 = pd.DataFrame(X_test_q2, columns=['question2'])
test_df = df_q1.join(df_q2, lsuffix='_left', rsuffix='_right')
test_df

,question1,question2
0,guys really think inexperienced women,men like inexperienced women
1,china disintegrate,china disintegrate like ussr
2,hardest adjustments make visiting georgia,hardest adjustments make visiting netherlands
3,cost starting global cable news network,best site read dc marvel comics free
4,find hyderabadi biryani chennai,cool ways say yes love hyderabad love hyderabadi
...,...,...
7999,get funding idea,get seed funding pre product
8000,top question ask coo plan business,anyone else feel compelled ask random question...
8001,would socially conservative fiscally liberal p...,dixiecrats fiscally liberal
8002,many people like jazz,many non korean people addicted kpop


# Using Spacy 

In [50]:
import nltk
from nltk.tokenize import word_tokenize  
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *

#p_stemmer = PorterStemmer()
#s_stemmer = SnowballStemmer(language='english')

def nltk_process(text):
    #Tokenization
    #nltk_tokenList = word_tokenize(text)
    
    #Stemming
    nltk_stemedList = []
    #for word in nltk_tokenList:
     #   nltk_stemedList.append(p_stemmer.stem(word))
        #nltk_stemedList.append(s_stemmer.stem(word))
    
    #Lemmatization
    wordnet_lemmatizer = WordNetLemmatizer()
    nltk_lemmaList = []
    for word in nltk_stemedList:
        nltk_lemmaList.append(wordnet_lemmatizer.lemmatize(word))
    
    #print("Stemming + Lemmatization")
    print(nltk_lemmaList)

#     #Filter stopword
#     filtered_sentence = []  
#     nltk_stop_words = set(stopwords.words("english"))
#     for w in nltk_lemmaList:  
#         if w not in nltk_stop_words:  
#             filtered_sentence.append(w)  

#     #Removing Punctuation
#     punctuations="?:!.,;"
#     for word in filtered_sentence:
#         if word in punctuations:
#             filtered_sentence.remove(word)
#     print(" ")
#     print("Remove stopword & Punctuation")
#     print(filtered_sentence)

In [53]:
#nltk_process(str(X_train_q1))

train_df['question1'] = train_df['question1'].apply(spacy_process)

NameError: name 'nlp' is not defined

## Labelling target variable

In [60]:
labels = preprocessing.LabelEncoder()
# Convert continous y values to categorical
y_train_cat= labels.fit_transform(y_train)
print(y_train_cat)

#y_val_cat = labels.fit_transform(y_val)
#print(y_val_cat)

y_test_cat = labels.fit_transform(y_test)
print(y_test_cat)

[1 0 1 ... 1 1 1]
[0 1 0 ... 0 0 1]


## Models 


In [268]:
from sklearn.feature_extraction.text import CountVectorizer

questions = list(train_df['question1']) + list(train_df['question2'])

vectorizer = CountVectorizer(analyzer='word')

X_train_q1_arr, X_train_q2_arr = np.vsplit(vectorizer.fit_transform(questions).toarray(), 2)

MemoryError: Unable to allocate 1.54 GiB for an array with shape (16170, 12792) and data type int64

## Text to Features - Count Vectorizer

In [63]:
from sklearn.feature_extraction.text import CountVectorizer

## X TRAIN QUESTION 1 ##

text = train_df['question1']

vectorizer = CountVectorizer()

vectorizer.fit(text) 
vector_X_train_q1 = vectorizer.transform(text)

print(vectorizer.vocabulary_)
print(vector_X_train_q1.toarray())

{'begin': 563, 'take': 5884, 'donald': 1745, 'trump': 6174, 'elected': 1893, 'president': 4600, 'think': 6013, 'may': 3622, 'actually': 75, 'good': 2496, 'many': 3561, 'legs': 3352, 'butterflies': 828, 'used': 6310, 'true': 6171, 'sweden': 5839, 'largest': 3300, 'number': 4044, 'rape': 4835, 'case': 898, 'europe': 2028, 'revenue': 5038, 'model': 3786, 'socialtrade': 5530, 'biz': 649, 'net': 3963, 'protector': 4696, 'antivirus': 281, 'comparison': 1189, 'quickheal': 4790, 'kaspersky': 3189, 'bitdefender': 646, 'use': 6309, 'recurrence': 4899, 'tree': 6154, 'determine': 1611, 'asymptotic': 395, 'upper': 6294, 'bound': 726, 'best': 593, 'way': 6503, 'evaluate': 2031, 'students': 5761, 'unable': 6231, 'ping': 4426, 'another': 271, 'computer': 1216, 'local': 3437, 'network': 3966, 'people': 4335, 'vomit': 6450, 'drinking': 1778, 'much': 3863, 'avoided': 457, 'history': 2702, 'port': 4515, 'blair': 655, 'andaman': 246, 'nicobar': 3989, 'island': 3066, 'india': 2889, 'real': 4861, 'world': 66

In [72]:
from sklearn.feature_extraction.text import CountVectorizer

text = train_df['question2']

vectorizer = CountVectorizer()

vectorizer.fit(text) 
vector_X_train_q2 = vectorizer.transform(text)

print(vectorizer.vocabulary_)
print(vector_X_train_q1.toarray())

{'donald': 1668, 'trump': 5913, 'potential': 4312, 'good': 2368, 'president': 4367, 'ok': 3914, 'chest': 986, 'legs': 3168, 'day': 1429, 'squat': 5364, 'use': 6040, 'leg': 3163, 'press': 4369, 'true': 5910, 'sweden': 5584, 'became': 537, 'rape': 4578, 'capital': 848, 'europe': 1926, 'revenue': 4783, 'model': 3598, 'openbay': 3934, 'urgent': 6031, 'net': 3775, 'protector': 4451, 'antivirus': 273, 'ran': 4569, 'windows': 6296, 'vulnerability': 6175, 'scan': 4949, 'showed': 5133, 'vulnerabilities': 6174, 'critical': 1355, 'important': 2733, 'moderate': 3601, 'option': 3949, 'anything': 280, 'repair': 4714, 'problem': 4394, 'shall': 5091, 'sure': 5567, 'even': 1935, 'means': 3462, 'recurrence': 4641, 'tree': 5889, 'determine': 1544, 'asymptotic': 390, 'upper': 6023, 'bound': 693, 'evaluate': 1929, 'people': 4115, 'computer': 1182, 'networking': 3778, 'network': 3777, 'computing': 1184, 'thing': 5751, 'difference': 1573, 'vomit': 6165, 'drinking': 1700, 'things': 5752, 'port': 4287, 'blair'

In [73]:
from sklearn.feature_extraction.text import CountVectorizer

test_text = test_df['question1']

vectorizer = CountVectorizer()

vectorizer.fit(test_text) 
vector_X_test_q1 = vectorizer.transform(test_text)

print(vectorizer.vocabulary_)
print(test_vector.toarray())

{'guys': 3849, 'really': 7271, 'think': 9002, 'inexperienced': 4373, 'women': 9870, 'china': 1529, 'disintegrate': 2532, 'hardest': 3904, 'adjustments': 139, 'make': 5296, 'visiting': 9623, 'georgia': 3652, 'cost': 2002, 'starting': 8473, 'global': 3695, 'cable': 1236, 'news': 5961, 'network': 5940, 'find': 3318, 'hyderabadi': 4157, 'biryani': 964, 'chennai': 1513, 'many': 5350, 'engineers': 2935, 'postmates': 6773, 'programmers': 6952, 'quora': 7150, 'started': 8471, 'old': 6168, 'languages': 4924, 'ever': 3041, 'good': 3731, 'without': 9858, 'learning': 4988, 'science': 7844, 'much': 5796, 'progress': 6956, 'scientists': 7849, 'made': 5258, 'bringing': 1144, 'back': 696, 'dead': 2231, 'people': 6491, 'overcome': 6299, 'heroin': 3991, 'addiction': 127, 'would': 9908, 'happen': 3891, 'earth': 2753, 'extra': 3147, 'moon': 5740, 'one': 6188, 'big': 925, 'phobos': 6565, 'english': 2939, 'words': 9883, 'used': 9469, 'conversations': 1952, 'indians': 4349, 'use': 9468, 'rare': 7221, 'exampl

In [76]:
from sklearn.feature_extraction.text import CountVectorizer

test_text = test_df['question2']

vectorizer = CountVectorizer()

vectorizer.fit(test_text) 
vector_X_test_q2 = vectorizer.transform(test_text)

print(vectorizer.vocabulary_)
print(test_vector.toarray())

{'men': 5333, 'like': 4930, 'inexperienced': 4238, 'women': 9444, 'china': 1459, 'disintegrate': 2410, 'ussr': 9064, 'hardest': 3795, 'adjustments': 125, 'make': 5130, 'visiting': 9201, 'netherlands': 5742, 'best': 849, 'site': 7837, 'read': 6989, 'dc': 2123, 'marvel': 5214, 'comics': 1644, 'free': 3367, 'cool': 1865, 'ways': 9310, 'say': 7490, 'yes': 9528, 'love': 5052, 'hyderabad': 4043, 'hyderabadi': 4044, 'many': 5176, 'engineers': 2814, 'pass': 6203, 'every': 2930, 'year': 9522, 'switzerland': 8403, 'ever': 2928, 'recognized': 7027, 'street': 8231, 'result': 7254, 'quora': 6902, 'popularity': 6523, 'science': 7528, 'capable': 1247, 'bringing': 1093, 'dead': 2127, 'back': 668, 'life': 4916, 'quit': 6897, 'heroin': 3891, 'addiction': 112, 'would': 9482, 'happen': 3782, 'earth': 2633, 'one': 5977, 'moon': 5551, 'approximately': 450, 'marks': 5197, 'score': 7540, 'get': 3529, 'rand': 6941, 'around': 490, 'srm': 8106, 'jee': 4544, 'scientology': 7535, 'scripture': 7560, 'mean': 5272, '

## N-GRAMS

In [87]:
from textblob import TextBlob

text = train_df['question1']

textblob = TextBlob(str(text)).ngrams(3)

textblob

[WordList(['0', 'begin', 'take']),
 WordList(['begin', 'take', 'donald']),
 WordList(['take', 'donald', 'trump']),
 WordList(['donald', 'trump', 'elected']),
 WordList(['trump', 'elected', 'president']),
 WordList(['elected', 'president', 'thin']),
 WordList(['president', 'thin', '1']),
 WordList(['thin', '1', 'many']),
 WordList(['1', 'many', 'legs']),
 WordList(['many', 'legs', 'butterflies']),
 WordList(['legs', 'butterflies', 'used']),
 WordList(['butterflies', 'used', '2']),
 WordList(['used', '2', 'true']),
 WordList(['2', 'true', 'sweden']),
 WordList(['true', 'sweden', 'largest']),
 WordList(['sweden', 'largest', 'number']),
 WordList(['largest', 'number', 'rape']),
 WordList(['number', 'rape', 'case']),
 WordList(['rape', 'case', 'europe']),
 WordList(['case', 'europe', '3']),
 WordList(['europe', '3', 'revenue']),
 WordList(['3', 'revenue', 'model']),
 WordList(['revenue', 'model', 'socialtrade']),
 WordList(['model', 'socialtrade', 'biz']),
 WordList(['socialtrade', 'biz', '

In [88]:
from sklearn.feature_extraction.text import HashVectorizer

text = train_df['question1']

vectorizer = HashingVectorizer(n_features=10)
vector= vectorizer.transform(text)
print(vector.shape)
print(vector.toarray())

ImportError: cannot import name 'HashVectorizer' from 'sklearn.feature_extraction.text' (C:\Users\43664\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py)

In [98]:
from sklearn.feature_extraction.text import TfidfVectorizer

text_q1 = train_df['question1']

vectorizer = TfidfVectorizer()

tfidf_X_train_q1 = vectorizer.fit_transform(text_q1)
print(tfidf_X_train_q1.shape)
#print(vectorizer.vocabulary_)
#print(vectorizer.idf_)






(4042, 6699)


In [99]:
from sklearn.feature_extraction.text import TfidfVectorizer

text_q2 = train_df['question2']

vectorizer = TfidfVectorizer()

tfidf_X_train_q2 = vectorizer.fit_transform(text_q2)
print(tfidf_X_train_q1.shape)
#print(vectorizer.vocabulary_)
#print(vectorizer.idf_)



(4042, 6699)


In [102]:
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(tfidf_X_train_q2[0:1], tfidf_X_train_q2)

#cosine_similarity(trsfm[0:1], trsfm)

array([[1., 0., 0., ..., 0., 0., 0.]])